In [ ]:
from block_ai.lib.myblokus import point
from block_ai.lib.myblokus.point import Point
from block_ai.lib.myblokus.piece import Piece
from block_ai.lib.myblokus.corner import Corner
from block_ai.lib.myblokus.orientation import Orientation

import itertools
import numpy as np
import random

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [ ]:
class Move:
    
    def __init__(self, orientation, player_id, piece_id, corner):
        self.orientation = orientation
        self.player_id = player_id
        self.piece_id = piece_id
        self.corner = corner

    def get_footprint(self):
        return [p for p in self.orientation.points]
    
    def __repr__(self):
        o_repr = repr(self.orientation)
        c_repr = repr(self.corner)
        return f"""Move(orientation={o_repr},
                        player_id={self.player_id},
                        piece_id='{self.piece_id}',
                        corner={c_repr})"""
    
    def __eq__(self, other):
        for attr in ["orientation", "player_id", "piece_id", "corner"]:
            if not eval(f"self.{attr} == other.{attr}"):
                return False
        return True

    def __str__(self):
        return f"Player ID: {self.player_id}\nPiece ID:{self.piece_id}\n{self.orientation}"

    
class Board:
    SIDE_LENGTH = 20
    EMPTY = -1
    
    def __init__(self):
        shape = (self.SIDE_LENGTH, self.SIDE_LENGTH)
        self.board = np.full(shape, self.EMPTY)
        
    def update(self, move):
        for p in move.get_footprint():
            self.assign(p, move.player_id)
        
    def are_squares_free(self, orientation):
        for p in orientation.points:
            if not self.is_square_free(p):
                return False
        return True

    def is_square_free(self, p):
        if not self.on_board(p):
            return False
        if not self.point_empty(p):
            return False
        return True

    @classmethod
    def on_board(self, point):
        valid_points = range(0, self.SIDE_LENGTH)
        return point.x in valid_points and point.y in valid_points
    
    def point_empty(self, p):
        val = self.check(p)
        logging.debug(val)
        return val == self.EMPTY
    
    def check(self, point):
        return self.board[point.y][point.x]
        
    def assign(self, point, value):
        if self.on_board(point):
            logging.debug("assigning %s: %s", point, value)
            self.board[point.y][point.x] = value
        else:
            logging.debug("%s off board not assigning", point)
            
    def __str__(self):
        return str(self.board)
    
    def copy(self):
        b = Board()
        b.board = self.board.copy()
        return b

    def display(self):
        vis = BoardView()
        vis.display(self.board)
        
def gen_pieces():
    pieces = {
        'p1': Piece([Point(0, 0)]),
        'p2': Piece([Point(0, 0), Point(0, 1)]),
        'p3': Piece([Point(0, 0), Point(0, 1), Point(0, 2)]),
        'p4': Piece([Point(0, 0), Point(0, 1), Point(1, 1)]),
        'p5': Piece([Point(0, 0), Point(1, 0), Point(0, 1), Point(1, 1)]),
        'p6': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0)]),
        'p7': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1)]),
        'p8': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 0)]),
        'p9': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1)]),
        'p10': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(4, 0)]),
        'p11': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(3, 1)]),
        'p12': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(2, 1), Point(3, 1)]),
        'p13': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(1, 1), Point(2, 1)]),
        'p14': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(0, 1), Point(2, 1)]),
        'p15': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(2, 1)]),
        'p16': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(1, 1), Point(1, 2)]),
        'p17': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(2, 1), Point(2, 2)]),
        'p18': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1), Point(2, 2)]),
        'p19': Piece([Point(0, 0), Point(0, 1), Point(1, 1), Point(2, 1), Point(2, 2)]),
        'p20': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1), Point(1, 2)]),
        'p21': Piece([Point(0, 1), Point(1, 1), Point(1, 2), Point(1, 0), Point(2, 1)])
    }
    
    return pieces

In [ ]:
"""Valid Moves store."""

class ValidMoves:
    
    def __init__(self):
        self.valid_moves = {}

    def add(self, move):
        try:
            corner_moves = self.valid_moves[move.corner]
            try:
                corner_moves[move.piece_id].append(move)
            except KeyError:
                corner_moves[move.piece_id] = [move]
        except KeyError:
            self.valid_moves[move.corner] = {}
            corner_moves = self.valid_moves[move.corner]
            corner_moves[move.piece_id] = [move]
    
    def remove(self, move):
        self.valid_moves[move.corner][move.piece_id].remove(move)
        
        if self.valid_moves[move.corner][move.piece_id] == []:
            del self.valid_moves[move.corner][move.piece_id]
    
        if self.valid_moves[move.corner] == {}:
            del self.valid_moves[move.corner]
        
    def get_all(self):
        gens = [self.get_corner_moves(c) for c in self.get_corners()]
        for m in itertools.chain(*gens):
            yield m

    def get_corner_moves(self, corner):
        for piece_id in self.valid_moves[corner]:
            for m in self.valid_moves[corner][piece_id]:
                yield m

    def get_corners(self):
        return self.valid_moves.keys()
    
    def get_corner_piece_moves(self, corner, piece_id):
        return filter(lambda m: m.piece_id == piece_id, self.get_corner_moves(corner))
            
    def get_piece_moves(self, piece_id):
        return filter(lambda m: m.piece_id == piece_id, self.get_valid_moves())

    def __len__(self):
        length = 0
        for val in self.valid_moves.values():
            for l in val.values():
                length += len(l)
        return length

In [ ]:
"""
This class defines a Player as a collection of.

Pieces, valid moves, and invalid squares.
"""

class Player:

    def __init__(self, player_id):
        self.player_id = player_id
        self.pieces = gen_pieces()
        self.valid_moves = ValidMoves()
        self.invalid_points = set()
        
    def update(self, move):
        logging.info("Updating player %s", self.player_id)
        
        if self.player_id == move.player_id:

            logging.info("Deleting piece id: %s", move.piece_id)
            del self.pieces[move.piece_id]
            
            logging.info("Adding border points")
            self.add_border_points(move)
            
        logging.info("Clearing moves")
        self.clear_moves(move)
        
       

    def add_border_points(self, move):
        invalid_points = move.orientation.get_border_points()
        invalid_points = list(filter(Board.on_board, invalid_points))
        self.invalid_points.update(invalid_points)

    def clear_moves(self, move):
        
        invalid_points = move.get_footprint()
        
        if self.player_id == move.player_id:
            invalid_points += move.orientation.get_border_points()
            
        valid_moves = list(self.valid_moves.get_all())

        for m in valid_moves:
            #if m == test_move:
            #    logger.setLevel(logging.DEBUG)
                
            if not self.is_move_valid(m):
                self.valid_moves.remove(m)
                continue
                
            if self.overlap(m, move):
                self.valid_moves.remove(m)
                
            #logger.setLevel(logging.INFO)

    def overlap(self, m1, m2):
        for p in m1.orientation.points:
            if p in m2.orientation.points:
                return True
        return False
            
    def is_move_valid(self, move):
        try:
            self.validate_move(move)
            return True
        except RuntimeError as err:
            logging.debug(err)
            return False

    def validate_move(self, move):
        if not self.has_piece(move.piece_id):
            raise RuntimeError(f"Move {move} invalid. Already played {move.piece_id}")

        new_points = {move.orientation.points}
        overlap = self.invalid_points.intersection(new_points)
        if len(overlap) > 0:
            raise RuntimeError(f"Move {move} invalid. Includes invalid point {p}")

    def has_piece(self, piece_id):
        return piece_id in self.pieces

    def get_corners(self):
        return self.valid_moves.get_corners()

    def add_move(self, move):
        self.valid_moves.add(move)

    def has_moves(self):
        return len(self.valid_moves) > 0

    def get_valid_moves(self):
        return self.valid_moves.get_all()

In [ ]:
"""A game of Blokus."""

class Game:

    def __init__(self):
        self.board = Board()
        self.player_pointer = 0
        self.players = [Player(i) for i in range(4)]
        self.set_starting_moves()
        self.move_history = []
        logging.info("Player Pointer: %s", self.player_pointer)
    
    def set_starting_moves(self):
        corners = [Corner(Point(-1, -1), Point(0, 0)),
                   Corner(Point(-1, 20), Point(0, 19)),
                   Corner(Point(20, 20), Point(19, 19)),
                   Corner(Point(20, -1), Point(19, 0))]

        for i, c in enumerate(corners):
            self.add_corner_moves(c, i)

    def make_move(self, move):
        logging.info("Making move: %s", move)
        if not move.player_id == self.player_pointer:
            raise RuntimeError(f"Move: {move} not players turn")

        try:
            self.validate_move(move)
            self.update_state(move)
            self.move_history.append(move)
            self.set_next_player()
        except RuntimeError as err:
            logging.exception(err)

    def update_state(self, move):
        self.board.update(move)
        
        for player in self.players:
            player.update(move)
            
        for corner in move.orientation.get_corners():
            self.add_corner_moves(corner, move.player_id)
 
    def add_corner_moves(self, corner, player_id):
        player = self.players[player_id]
        rotation = corner.get_rotation()
        
        if not Board.on_board(corner.p2):
            logging.info("Corner %s is off board", corner)
            return
        
        logging.info("Adding corner %s moves", corner)
        for piece_id, piece in player.pieces.items():
            
            for orientation in piece.orientations:
                logging.debug("Examining orientation: %s", orientation)
                
                new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
                logging.debug("New orientation: %s", new_o)
                m = Move(new_o, player_id, piece_id, corner)

                if self.is_move_valid(m):
                    logging.debug("Adding Move: %s", m)
                    player.add_move(m)

    def is_move_valid(self, move):
        try:
            self.validate_move(move)
            return True
        except RuntimeError as err:
            logging.debug(err)
            return False
    
    def validate_move(self, move):
        
        if not self.board.are_squares_free(move.orientation):
            raise RuntimeError(f"Move: {move} squares are not free")

        self.players[move.player_id].validate_move(move)
    
    
    def set_next_player(self):
        logging.info("Advancing player")
        self.player_pointer = self.get_next_player()

    def get_next_player(self):
        ptr = self.player_pointer
        logging.info("starting at %s", ptr)
        for i in range(4):
            ptr = self.advance(ptr)
            if self.players[ptr].has_moves():
                logging.info("returning %s", ptr)
                return ptr
        raise GameEnd("Game Over")
        
    
    def advance(self, player_id):
        return (player_id + 1) % 4

    def get_players_moves(self, player_id):
        return self.players[player_id].get_valid_moves()
    
    def display(self, player_id=None, bad_move=None):
        board = self.board
        
        if player_id is not None:
            board = self.fill_player_pov(board, player_id)
            
        if bad_move is not None:
            board = self.fill_bad_move(board, bad_move)

        board.display()

    def fill_player_pov(self, board, player_id):
        board = board.copy()
        
        adjacent_fill = 4
        corner_fill = 5
        
        for p in self.players[player_id].invalid_points:
            if board.point_empty(p):
                board.assign(p, adjacent_fill)
            
        for corner in self.players[player_id].get_corners():
            if board.point_empty(corner.p2):
                board.assign(corner.p2, corner_fill)
        return board

    def fill_bad_move(self, board, move):
        bad_fill = 6
        
        board = board.copy()

        for p in move.orientation.points:
            board.assign(p, bad_fill)
        return board
    
class GameEnd(Exception):
    pass

## Visualization Code

In [ ]:
class BoardView:

    def __init__(self):
        pass
    
    def display(self, board):
        n = 20
        my_dpi = 20
        canvasSize = (n,n)
        fig,ax = self.createCanvas(canvasSize[0],canvasSize[1],my_dpi)
        color = "black"

        for row in range(n):
            for col in range(n):
                color = self.get_color(board[row][col])
                rect = self.createRectangle((1,1),(col,row),canvasSize,color)
                ax.add_patch(rect)

        plt.show()

    def createCanvas(self, width,height,my_dpi):
        fig,ax = plt.subplots(1)
        fig.dpi= my_dpi
        fig.set_size_inches(width,height)
        ax.set_xticks([])
        ax.set_yticks([])
        return fig,ax

    def createRectangle(self, wh, xy, cwch, color="black"):
        """
        All units are in inches. 

        Parameters
        ==========
        wh: (width,height) 
        xy: (x,y)
        cwch: (canvasWidth,canvasHeight)
        """
        return patches.Rectangle((xy[0]/cwch[0],xy[1]/cwch[1]),wh[0]/cwch[0],wh[1]/cwch[1],color=color)

    def get_color(self, val):
        mapping = {
            -1: "#DCDCDC",
            0: "yellow",
            1: "red",
            2: "green",
            3: "blue",
            4: "orange",
            5: "purple",
            6: "black"
        }
        return mapping[val]

In [ ]:
class GameEngine:
    
    def __init__(self, display=False):
        self.display = display
        self.game = Game()
        self.players = [RandomAgent(i) for i in range(4)]
        
    
    def play_game(self):
        while True:
            try:
                self.play_turn()
            except GameEnd:
                break
        
    def play_turn(self):
        try:
            m = self.get_move()
            self.game.make_move(m)
        except RuntimeError as err:
            logging.info(err)
            self.game.set_next_player()

        if self.display:
            self.game.display(self.game.player_pointer - 1)
            
    def get_move(self):
        ptr = self.game.player_pointer
        logging.info("Player pointer: %s", ptr)
        agent = self.players[ptr]
        logging.info("Agent id: %s", agent.player_id)
        return agent.get_move(self.game)

In [ ]:
class Agent:

    def __init__(self, player_id):
        self.player_id = player_id
    
    def get_move(self, game):
        raise RuntimeError("get_move is not implimented")
    
    
class RandomAgent(Agent):
    
    def __init__(self, player_id):
        self.player_id = player_id
        
    def get_move(self, game):
        valid_moves = list(game.get_players_moves(self.player_id))
        return self.pick_random_move(valid_moves)
        
    def pick_random_move(self, moves):     
        if len(moves) == 0:
            raise RuntimeError(f"Player {self.player_id} is out of moves")

        rand_int = random.randrange(len(moves))
        return moves[rand_int]

In [ ]:
logger.setLevel(logging.WARNING)

50% of get corner moves involves rotating points. Are matracies faster?

Nagging Ideas:
     -  I should set the seeds for random players to or replay a game based on move history
        so that the runtimes are the same
    
     -  Separate classes into files w/ tests so I don't break anything making optimizations

     -  Speed up BoardViewer 

Optimization Ideas:
     - lots of runtime spend rotating points. Are np.arrays / rot methods faster?
     
     - a bunch of time is spent converting orientations to sets in player.validate_move
         - raise try / catch is pretty expensive though it's nice to have reason why move is invalid
         - could use a boolean check + explain menthod (a little ugly)
  
     - logging can slow down stuff remove ones from the inner most loops

Fixed Logging

File: <ipython-input-74-8f1a337fe857>
Function: add_corner_moves at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    45                                               def add_corner_moves(self, corner, player_id):
    46       336        358.0      1.1      0.0          player = self.players[player_id]
    47       336       3075.0      9.2      0.2          rotation = corner.get_rotation()
    48                                                   
    49       336        946.0      2.8      0.1          if not Board.on_board(corner.p2):
    50        45        159.0      3.5      0.0              logging.info("Corner %s is off board", corner)
    51        45         18.0      0.4      0.0              return
    52                                                   
    53       291       1214.0      4.2      0.1          logging.info("Adding corner %s moves", corner)
    54      4006       2874.0      0.7      0.2          for piece_id, piece in player.pieces.items():
    55                                                       
    56     20724      13778.0      0.7      0.8              for orientation in piece.orientations:
    57     17009      50509.0      3.0      3.0                  logging.debug("Examining orientation: %s", orientation)
    58                                                           
    59     17009     882254.0     51.9     52.7                  new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
    60     17009      59013.0      3.5      3.5                  logging.debug("New orientation: %s", new_o)
    61     17009      33927.0      2.0      2.0                  m = Move(new_o, player_id, piece_id, corner)
    62                                           
    63     17009     567201.0     33.3     33.9                  if self.is_move_valid(m):
    64      6450      21146.0      3.3      1.3                      logging.debug("Adding Move: %s", m)
    65      6450      36560.0      5.7      2.2                      player.add_move(m)

Timer unit: 1e-06 s

Total time: 1.43679 s
File: <ipython-input-54-6bf5e323533c>
Function: add_corner_moves at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    45                                               def add_corner_moves(self, corner, player_id):
    46       354        281.0      0.8      0.0          player = self.players[player_id]
    47       354       2668.0      7.5      0.2          rotation = corner.get_rotation()
    48                                                   
    49       354        796.0      2.2      0.1          if not Board.on_board(corner.p2):
    50        41        127.0      3.1      0.0              logging.info("Corner %s is off board", corner)
    51        41         19.0      0.5      0.0              return
    52                                                   
    53       313       1007.0      3.2      0.1          logging.info("Adding corner %s moves", corner)
    54      4174       2446.0      0.6      0.2          for piece_id, piece in player.pieces.items():
    55                                                       
    56     20946      10877.0      0.5      0.8              for orientation in piece.orientations:
    57     17085      43595.0      2.6      3.0                  logging.debug("Examining orientation: %s", orientation)
    58                                                           
    59     17085     716453.0     41.9     49.9                  new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
    60     17085      46508.0      2.7      3.2                  logging.debug("New orientation: %s", new_o)
    61     17085      49656.0      2.9      3.5                  m = Move(new_o, player_id, piece_id, corner)
    62                                           
    63     17085     454759.0     26.6     31.7                  if self.is_move_valid(m):
    64      6220      78904.0     12.7      5.5                      logging.debug("Adding Move: %s", indent(m, 1))
    65      6220      28691.0      4.6      2.0                      player.add_move(m)

Timer unit: 1e-06 s

Total time: 1.43679 s
File: <ipython-input-54-6bf5e323533c>
Function: add_corner_moves at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    45                                               def add_corner_moves(self, corner, player_id):
    46       354        281.0      0.8      0.0          player = self.players[player_id]
    47       354       2668.0      7.5      0.2          rotation = corner.get_rotation()
    48                                                   
    49       354        796.0      2.2      0.1          if not Board.on_board(corner.p2):
    50        41        127.0      3.1      0.0              logging.info("Corner %s is off board", corner)
    51        41         19.0      0.5      0.0              return
    52                                                   
    53       313       1007.0      3.2      0.1          logging.info("Adding corner %s moves", corner)
    54      4174       2446.0      0.6      0.2          for piece_id, piece in player.pieces.items():
    55                                                       
    56     20946      10877.0      0.5      0.8              for orientation in piece.orientations:
    57     17085      43595.0      2.6      3.0                  logging.debug("Examining orientation: %s", orientation)
    58                                                           
    59     17085     716453.0     41.9     49.9                  new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
    60     17085      46508.0      2.7      3.2                  logging.debug("New orientation: %s", new_o)
    61     17085      49656.0      2.9      3.5                  m = Move(new_o, player_id, piece_id, corner)
    62                                           
    63     17085     454759.0     26.6     31.7                  if self.is_move_valid(m):
    64      6220      78904.0     12.7      5.5                      logging.debug("Adding Move: %s", indent(m, 1))
    65      6220      28691.0      4.6      2.0                      player.add_move(m)

In [ ]:
ge = GameEngine(display=False)

%lprun -f ge.game.add_corner_moves ge.play_game()
ge.play_game()

In [ ]:
ge = GameEngine(display=False)

%lprun -f Player.validate_move ge.play_game()
ge.play_game()

```Third Run
Changed to set intersection instead itterative check almost the same time requirement```

Total time: 0.526981 s
File: <ipython-input-53-8fbb860b920e>
Function: validate_move at line 72

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    72                                               def validate_move(self, move):
    73     72684      72960.0      1.0     13.8          if not self.has_piece(move.piece_id):
    74      1540      17668.0     11.5      3.4              raise RuntimeError(f"Move {move} invalid. Already played {move.piece_id}")
    75                                           
    76     71144     350042.0      4.9     66.4          new_points = {move.orientation.points}
    77     71144      49508.0      0.7      9.4          overlap = self.invalid_points.intersection(new_points)
    78     71144      36803.0      0.5      7.0          if len(overlap) > 0:
    79                                                       raise RuntimeError(f"Move {move} invalid. Includes invalid point {p}")

```Second Run
Used set instead list to hold invalid points```

Total time: 0.526422 s
File: <ipython-input-44-b012ab35adb8>
Function: validate_move at line 72

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    72                                               def validate_move(self, move):
    73     48991      56822.0      1.2     10.8          if not self.has_piece(move.piece_id):
    74       833       9379.0     11.3      1.8              raise RuntimeError(f"Move {move} invalid. Already played {move.piece_id}")
    75                                           
    76    247885      96865.0      0.4     18.4          for p in move.orientation.points:
    77    204076     302546.0      1.5     57.5              if p in self.invalid_points:
    78      4349      60810.0     14.0     11.6                  raise RuntimeError(f"Move {move} invalid. Includes invalid point {p}")

```First Run```

Timer unit: 1e-06 s

Total time: 8.30674 s
File: <ipython-input-7-e9cda3e2ec9f>
Function: validate_move at line 72

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    72                                               def validate_move(self, move):
    73     48443      74686.0      1.5      0.9          if not self.has_piece(move.piece_id):
    74       715      13962.0     19.5      0.2              raise RuntimeError(f"Move {move} invalid. Already played {move.piece_id}")
    75                                           
    76    239866     157205.0      0.7      1.9          for p in move.orientation.points:
    77    196660    7988919.0     40.6     96.2              if p in self.invalid_points:
    78      4522      71967.0     15.9      0.9                  raise RuntimeError(f"Move {move} invalid. Includes invalid point {p}")

## Status
Refactored but a bunch of cornsers get droped for no reason near the end

#### Questions:

why does it allow invalid moves in the below loop?

Theres an average branching factor of around 25
so call it 75 is that doable?





Number of chess game one guy says with 40 avg moves avg 30 moves per game
~ 10 ** 120 

James Hardy says (10 ** 50) ** 50 

Number of blockus game???

In [ ]:
f"{len(str(75 ** (20 * 4)))}"

In [ ]:

products = []

for i in range(100):
    
    if i > 0 and i % 10 == 0:
        print(i)
        
    ge = GameEngine()
    move_counts = []
    while True:
        try:
            player_id = ge.game.player_pointer
            moves = list(ge.game.get_players_moves(player_id))
            move_counts.append((player_id, len(moves)))
            ge.play_turn()

        except GameEnd:
            break

    ids, options = zip(*move_counts)
    options = [float(o) for o in options]
    product = np.product(options)
    products.append(product)

